In [1]:
import os
from omegaconf import DictConfig

In [2]:
files_list: list[dict[str, str | list]] = []
files_list

[]

In [3]:
configs = [
    ['/home/sha/work/yi/data-scripts-queries/ecs', '/home/sha/work/yi/data-scripts-queries/queries/query_ecs_yi.txt', 'ecs'],
    ['/home/sha/work/yi/data-scripts-queries/pwy', '/home/sha/work/yi/data-scripts-queries/queries/query_pwy_yi.txt', 'pwy'],
    ['/home/sha/work/yi/data-scripts-queries/tax', '/home/sha/work/yi/data-scripts-queries/queries/query_tax_yi.txt', 'tax'],
]

In [4]:
for item in configs:
    if item[0] is not None and item[1] is not None:
            files_list.append({"name": item[2], "path": item[2], "files": []})
    else:
        pass

In [5]:
# files_list

In [8]:
queries: dict[str, str] = {}
# with open('/home/sha/work/yi/data-scripts-queries/queries/query_meta.txt', "r") as file:
#     queries["master_data"] = file.read()
with open('/home/sha/work/yi/data-scripts-queries/queries/query_ecs_yi.txt', "r") as file:
    queries["ecs"] = file.read()
with open('/home/sha/work/yi/data-scripts-queries/queries/query_pwy_yi.txt', "r") as file:
    queries["pwy"] = file.read()
with open('/home/sha/work/yi/data-scripts-queries/queries/query_tax_yi.txt', "r") as file:
    queries["tax"] = file.read()

In [9]:
queries

{'ecs': 'LOAD CSV WITH HEADERS FROM $path AS row\nMERGE (gen:Genus {name: row.Genus})\nMERGE (enz:Enzyme {name: row.Enzyme})\nCREATE (gen)-[rel: Related {abundance_mean: row.mean, classification: row.classification, group: row.group}]->(enz)',
 'pwy': 'LOAD CSV WITH HEADERS FROM $path AS row\nMERGE (gen:Genus {name: row.Genus})\nMERGE (pwy:Pathway {name: row.PWY})\nCREATE (gen)-[rel: Related {abundance_mean: row.mean, classification: row.classification, group: row.group}]->(pwy)',
 'tax': "LOAD CSV WITH HEADERS FROM $path AS row\nMERGE (diag:Diagnosis {name: row.diagnosis})\nMERGE (gen:Genus {name: row.genus})\nCREATE (diag)<-[rel:Presence {classification: row.classification, group: row.group, abundance_mean: row.mean, significant_difference: CASE WHEN row.contrasts IS NULL THEN [] ELSE split(row.contrasts, '|') END}]-(gen)\n"}

In [10]:
queries_store = []
# queries_store.append(
#     [
#         queries["master_data"],
#         "file:///" + 'data/meta/master_data.csv',
#         "Importing Master Data",
#     ]
# )
neo4j_import_folder = '/home/sha/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-c7db64bb-a3cf-43a4-af5c-7438bcb4790d/import/data_yi'
for file_item in files_list:
    print(file_item)
    if os.path.exists(
        str(neo4j_import_folder) + "/" + str(file_item["path"])
    ):
        file_item["files"] = os.listdir(
            neo4j_import_folder + "/" + file_item["path"]
        )

        for file in file_item["files"]:
            queries_store.append(
                [
                    queries[file_item["name"]],
                    f"file:///data_yi/{file_item['path']}/{file}",
                    f"Importing {file_item['name']} data from {file}",
                ]
            )

{'name': 'ecs', 'path': 'ecs', 'files': []}
{'name': 'pwy', 'path': 'pwy', 'files': []}
{'name': 'tax', 'path': 'tax', 'files': []}


In [11]:
queries_store

[['LOAD CSV WITH HEADERS FROM $path AS row\nMERGE (gen:Genus {name: row.Genus})\nMERGE (enz:Enzyme {name: row.Enzyme})\nCREATE (gen)-[rel: Related {abundance_mean: row.mean, classification: row.classification, group: row.group}]->(enz)',
  'file:///data_yi/ecs/ecs_age_group_out.csv',
  'Importing ecs data from ecs_age_group_out.csv'],
 ['LOAD CSV WITH HEADERS FROM $path AS row\nMERGE (gen:Genus {name: row.Genus})\nMERGE (enz:Enzyme {name: row.Enzyme})\nCREATE (gen)-[rel: Related {abundance_mean: row.mean, classification: row.classification, group: row.group}]->(enz)',
  'file:///data_yi/ecs/ecs_visit_stage_out.csv',
  'Importing ecs data from ecs_visit_stage_out.csv'],
 ['LOAD CSV WITH HEADERS FROM $path AS row\nMERGE (gen:Genus {name: row.Genus})\nMERGE (enz:Enzyme {name: row.Enzyme})\nCREATE (gen)-[rel: Related {abundance_mean: row.mean, classification: row.classification, group: row.group}]->(enz)',
  'file:///data_yi/ecs/ecs_Sex_out.csv',
  'Importing ecs data from ecs_Sex_out.csv'

In [12]:
# from neo4j import GraphDatabase
# driver = GraphDatabase.driver(uri, auth=(user, password))
# conn = Neo4jConnection(
#         config.neo4j_config.url, config.neo4j_config.user, config.neo4j_config.pw
#     )
# for query in queries_store:
#     conn.query(query[0], path=query[1])
#     with driver.session() as session:
#         response = list(session.run(query, **kwargs))
#         return response
# driver.close()

In [13]:
from neo4j_connection import Neo4jConnection

In [14]:
from neo4j import GraphDatabase

In [15]:
NEO4J = {"URL": "bolt://localhost:7687", "USER": "neo4j", "PW": "1qazXSW@", 'DB': 'ecspwytax'}
URI = NEO4J['URL']
AUTH = (NEO4J['USER'], NEO4J['PW'])

with GraphDatabase.driver(URI, auth=AUTH) as driver: 
    driver.verify_connectivity()
    for query in queries_store:
        print(f"processing {query[1]}")
        driver.execute_query(query[0], path=query[1], database_=NEO4J['DB'])

processing file:///data_yi/ecs/ecs_age_group_out.csv
processing file:///data_yi/ecs/ecs_visit_stage_out.csv
processing file:///data_yi/ecs/ecs_Sex_out.csv
processing file:///data_yi/pwy/pwy_visit_stage_out.csv
processing file:///data_yi/pwy/pwy_Sex_out.csv
processing file:///data_yi/pwy/pwy_age_group_out.csv
processing file:///data_yi/tax/tax_age_group_out.csv
processing file:///data_yi/tax/tax_visit_stage_out.csv
processing file:///data_yi/tax/tax_Sex_out.csv


In [13]:
NEO4J = {"URL": "bolt://localhost:7687", "USER": "neo4j", "PW": "1qazXSW@"}

neo4j_config_url = NEO4J['URL']
neo4j_config_user = NEO4J['USER']
neo4j_config_pw = NEO4J['PW']
conn = Neo4jConnection(neo4j_config_url, neo4j_config_user, neo4j_config_pw, 'biocypher')
for query in queries_store:
    print(f"processing {query[1]}")
    conn.query(query[0], path=query[1])
conn.close()

processing file:///data/meta/master_data.csv
processing file:///data_yi/ecs/ecs_age_group_out.csv
processing file:///data_yi/ecs/ecs_visit_stage_out.csv
processing file:///data_yi/ecs/ecs_Sex_out.csv
processing file:///data_yi/pwy/pwy_visit_stage_out.csv
processing file:///data_yi/pwy/pwy_Sex_out.csv
processing file:///data_yi/pwy/pwy_age_group_out.csv
processing file:///data_yi/tax/tax_age_group_out.csv
processing file:///data_yi/tax/tax_visit_stage_out.csv
processing file:///data_yi/tax/tax_Sex_out.csv
